In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 1. Загрузка данных

In [3]:
data_dir = '../data/'
processed_dir = '../data/processed'
os.makedirs(processed_dir, exist_ok=True)

In [4]:
anime_path = os.path.join(data_dir, 'anime-dataset-2023.csv')
users_score_path = os.path.join(data_dir, 'users-score-2023.csv')
users_details_path = os.path.join(data_dir, 'users-details-2023.csv')

In [5]:
processed_anime_path = os.path.join(processed_dir, 'anime_processed.csv')
processed_scores_path = os.path.join(processed_dir, 'users_score_processed.csv')
processed_details_path = os.path.join(processed_dir, 'users_details_processed.csv')
merged_path = os.path.join(processed_dir, 'merged_scores_details.csv')

In [6]:
chunksize = 1000000  # 1M строк за раз

In [7]:
# Anime DF (маленький, загружаем полностью)
# if os.path.exists(processed_anime_path):
#     anime_df = pd.read_csv(processed_anime_path)
#     print(f"Загружен processed anime: {anime_df.shape}")
# else:
anime_df = pd.read_csv(anime_path)
print(f"Загружен исходный anime: {anime_df.shape}")

Загружен исходный anime: (24905, 24)


In [8]:
# Users Score DF (большой, chunks если не processed)
# if os.path.exists(processed_scores_path):
#     users_score_df = pd.read_csv(processed_scores_path)
#     print(f"Загружен processed users-score: {users_score_df.shape}")
# else:
chunks = pd.read_csv(users_score_path, chunksize=chunksize)
processed_chunks = []
for chunk in chunks:
    processed_chunks.append(chunk)
users_score_df = pd.concat(processed_chunks)
print(f"Загружен исходный users-score: {users_score_df.shape}")

Загружен исходный users-score: (24325191, 5)


In [9]:
# Users Details DF (средний, full)
    # if os.path.exists(processed_details_path):
#     users_details_df = pd.read_csv(processed_details_path)
#     print(f"Загружен processed users-details: {users_details_df.shape}")
# else:
users_details_df = pd.read_csv(users_details_path)
print(f"Загружен исходный users-details: {users_details_df.shape}")

Загружен исходный users-details: (731290, 16)


In [10]:
# Переименование 'Mal ID' в 'user_id' для users_details_df
if 'Mal ID' in users_details_df.columns and 'user_id' not in users_details_df.columns:
    users_details_df.rename(columns={'Mal ID': 'user_id'}, inplace=True)
    print("Переименована колонка 'Mal ID' в 'user_id' в users_details_df.")

Переименована колонка 'Mal ID' в 'user_id' в users_details_df.


In [11]:
# Merged DF (если не processed)
# if os.path.exists(merged_path):
#     merged_scores_details = pd.read_csv(merged_path, parse_dates=['Joined'])
#     print(f"Загружен processed merged: {merged_scores_details.shape}")
# else:
merged_scores_details = pd.merge(users_score_df, users_details_df, on='user_id', how='left')
print(f"Создан исходный merged: {merged_scores_details.shape}")

Создан исходный merged: (24325191, 20)


# 2. Предварительный обзор данных

In [12]:
# Обзор anime_df
print("Anime DF info:")
anime_df.info()
print("\nБазовые статистики:")
print(anime_df.describe())
print("\nКатегориальные статистики:")
print(anime_df.describe(include='object'))
print("\nПервые 5 строк:")
print(anime_df.head())
print("\nПропуски:")
print(anime_df.isnull().sum())
print("\nДубликаты:", anime_df.duplicated().sum())

Anime DF info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24905 entries, 0 to 24904
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   anime_id      24905 non-null  int64 
 1   Name          24905 non-null  object
 2   English name  24905 non-null  object
 3   Other name    24905 non-null  object
 4   Score         24905 non-null  object
 5   Genres        24905 non-null  object
 6   Synopsis      24905 non-null  object
 7   Type          24905 non-null  object
 8   Episodes      24905 non-null  object
 9   Aired         24905 non-null  object
 10  Premiered     24905 non-null  object
 11  Status        24905 non-null  object
 12  Producers     24905 non-null  object
 13  Licensors     24905 non-null  object
 14  Studios       24905 non-null  object
 15  Source        24905 non-null  object
 16  Duration      24905 non-null  object
 17  Rating        24905 non-null  object
 18  Rank          24905 non-null  o

In [13]:
# Обзор users_score_df
print("Users Score DF info:")
users_score_df.info()
print("\nБазовые статистики:")
print(users_score_df.describe())
print("\nПропуски:")
print(users_score_df.isnull().sum())
print("\nДубликаты:", users_score_df.duplicated().sum())

Users Score DF info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24325191 entries, 0 to 24325190
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   user_id      int64 
 1   Username     object
 2   anime_id     int64 
 3   Anime Title  object
 4   rating       int64 
dtypes: int64(3), object(2)
memory usage: 927.9+ MB

Базовые статистики:
            user_id      anime_id        rating
count  2.432519e+07  2.432519e+07  2.432519e+07
mean   4.403843e+05  9.754686e+03  7.622930e+00
std    3.669469e+05  1.206196e+04  1.661510e+00
min    1.000000e+00  1.000000e+00  1.000000e+00
25%    9.718800e+04  8.730000e+02  7.000000e+00
50%    3.879780e+05  4.726000e+03  8.000000e+00
75%    5.280430e+05  1.316100e+04  9.000000e+00
max    1.291097e+06  5.608500e+04  1.000000e+01

Пропуски:
user_id          0
Username       232
anime_id         0
Anime Title      0
rating           0
dtype: int64

Дубликаты: 0


In [14]:
# Обзор users_details_df
print("Users Details DF info:")
users_details_df.info()
print("\nБазовые статистики:")
print(users_details_df.describe())
print("\nПропуски:")
print(users_score_df.isnull().sum())
print("\nДубликаты:", users_score_df.duplicated().sum())

Users Details DF info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731290 entries, 0 to 731289
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           731290 non-null  int64  
 1   Username          731289 non-null  object 
 2   Gender            224383 non-null  object 
 3   Birthday          168068 non-null  object 
 4   Location          152805 non-null  object 
 5   Joined            731290 non-null  object 
 6   Days Watched      731282 non-null  float64
 7   Mean Score        731282 non-null  float64
 8   Watching          731282 non-null  float64
 9   Completed         731282 non-null  float64
 10  On Hold           731282 non-null  float64
 11  Dropped           731282 non-null  float64
 12  Plan to Watch     731282 non-null  float64
 13  Total Entries     731282 non-null  float64
 14  Rewatched         731282 non-null  float64
 15  Episodes Watched  731282 non-null  float64
dt

In [15]:
# Обзор merged_scores_details
print("Merged DF info:")
merged_scores_details.info()
print("\nБазовые статистики:")
print(merged_scores_details.describe())
print("\nПервые 5 строк:")
print(merged_scores_details.head())
print("\nПропуски:")
print(merged_scores_details.isnull().sum())
print("\nДубликаты:", merged_scores_details.duplicated().sum())

Merged DF info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24325191 entries, 0 to 24325190
Data columns (total 20 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           int64  
 1   Username_x        object 
 2   anime_id          int64  
 3   Anime Title       object 
 4   rating            int64  
 5   Username_y        object 
 6   Gender            object 
 7   Birthday          object 
 8   Location          object 
 9   Joined            object 
 10  Days Watched      float64
 11  Mean Score        float64
 12  Watching          float64
 13  Completed         float64
 14  On Hold           float64
 15  Dropped           float64
 16  Plan to Watch     float64
 17  Total Entries     float64
 18  Rewatched         float64
 19  Episodes Watched  float64
dtypes: float64(10), int64(3), object(7)
memory usage: 3.6+ GB

Базовые статистики:
            user_id      anime_id        rating  Days Watched    Mean Score  \
count  2.432519e+07  2.4

# 3. Предобработка данных

In [22]:
# Обработка anime_df
anime_df = anime_df[anime_df['Popularity'] > 0]
anime_df['Genres'] = anime_df['Genres'].fillna('Unknown')
# Сначала преобразуем 'Score' в numeric, 'UNKNOWN' -> NaN
anime_df['Score'] = pd.to_numeric(anime_df['Score'], errors='coerce')
# Теперь median на non-NaN и fill
median_score = anime_df['Score'].median()
anime_df['Score'] = anime_df['Score'].fillna(median_score)
anime_df = anime_df.dropna(subset=['anime_id', 'Name'])
anime_df = anime_df.drop_duplicates(subset=['anime_id'])
anime_df['Episodes'] = pd.to_numeric(anime_df['Episodes'], errors='coerce')
anime_df['Episodes'] = anime_df['Episodes'].fillna(1)
# Для Aired: Преобразовать в str, парсить start date из ranges
anime_df['Aired'] = anime_df['Aired'].astype(str).str.split(' to ').str[0]
anime_df['Aired'] = pd.to_datetime(anime_df['Aired'], errors='coerce')
# Преобразование Rank и Scored By в numeric (аналогично Score)
anime_df['Rank'] = pd.to_numeric(anime_df['Rank'], errors='coerce')
anime_df['Rank'] = anime_df['Rank'].fillna(anime_df['Rank'].median())
anime_df['Scored By'] = pd.to_numeric(anime_df['Scored By'], errors='coerce')
anime_df['Scored By'] = anime_df['Scored By'].fillna(anime_df['Scored By'].median())

In [17]:
# Обработка users_score_df
users_score_df = users_score_df[users_score_df['rating'] > 0]
users_score_df['rating'] = users_score_df['rating'].fillna(users_score_df['rating'].median())
users_score_df = users_score_df.drop_duplicates(subset=['user_id', 'anime_id'])
# Обработка пропусков в Username
users_score_df['Username'] = users_score_df['Username'].fillna('Unknown')

In [18]:
# Обработка users_details_df
# Обработка числовых колонок с пропусками (замена медианой и приведение к числу)
for col in ['Days Watched', 'Mean Score', 'Completed', 'Watching', 'On Hold', 'Dropped', 'Plan to Watch', 'Total Entries', 'Rewatched', 'Episodes Watched']:
    users_details_df[col] = users_details_df[col].fillna(users_details_df[col].median())
    users_details_df[col] = pd.to_numeric(users_details_df[col], errors='coerce')

# Удаление дубликатов по user_id
users_details_df = users_details_df.drop_duplicates(subset=['user_id'])

# Преобразование Joined в datetime (обработка формата с T и +00:00)
users_details_df['Joined'] = pd.to_datetime(users_details_df['Joined'].astype(str).str.replace('T00:00:00\+00:00', '', regex=True), errors='coerce')

# Обработка пропусков в категориальных колонках
users_details_df['Username'] = users_details_df['Username'].fillna('Unknown')
users_details_df['Gender'] = users_details_df['Gender'].fillna('Unknown')
users_details_df['Location'] = users_details_df['Location'].fillna('Unknown')
users_details_df['Birthday'] = users_details_df['Birthday'].fillna('Unknown')

In [19]:
# Обработка merged_scores_details с использованием chunks
chunksize = 1000000
merged_chunks = []
for chunk in pd.read_csv(users_score_path, chunksize=chunksize):  # Перечитываем исходный файл порциями
    chunk = chunk[chunk['rating'] > 0]
    chunk['rating'] = chunk['rating'].fillna(chunk['rating'].median())
    chunk = chunk.drop_duplicates(subset=['user_id', 'anime_id'])
    chunk['Username'] = chunk['Username'].fillna('Unknown')
    # Объединяем текущую порцию с users_details_df
    merged_chunk = pd.merge(chunk, users_details_df, on='user_id', how='left')
    merged_chunks.append(merged_chunk)

# Объединяем все порции в один DataFrame
merged_scores_details = pd.concat(merged_chunks, ignore_index=True)

# Дополнительная обработка merged_scores_details
merged_scores_details = merged_scores_details.dropna(subset=['user_id', 'anime_id'])
for col in ['Days Watched', 'Mean Score', 'Completed', 'Watching', 'On Hold', 'Dropped', 'Plan to Watch', 'Total Entries', 'Rewatched', 'Episodes Watched']:
    merged_scores_details[col] = merged_scores_details[col].fillna(merged_scores_details[col].median())
    merged_scores_details[col] = pd.to_numeric(merged_scores_details[col], errors='coerce')
merged_scores_details['Username_x'] = merged_scores_details['Username_x'].fillna('Unknown')
merged_scores_details['Username_y'] = merged_scores_details['Username_y'].fillna('Unknown')
merged_scores_details['Gender'] = merged_scores_details['Gender'].fillna('Unknown')
merged_scores_details['Location'] = merged_scores_details['Location'].fillna('Unknown')
merged_scores_details['Birthday'] = merged_scores_details['Birthday'].fillna('Unknown')
merged_scores_details['Joined'] = pd.to_datetime(merged_scores_details['Joined'].astype(str).str.replace('T00:00:00\+00:00', '', regex=True), errors='coerce')
merged_scores_details = merged_scores_details.drop_duplicates(subset=['user_id', 'anime_id'])

In [20]:
# Расширенная статистика после предобработки
print("=== Статистика после предобработки ===")

print("\nAnime DF:")
print("Shape:", anime_df.shape)
anime_df.info()
print("\nБазовые статистики (числовые):")
print(anime_df.describe())
print("\nКатегориальные статистики:")
print(anime_df.describe(include='object'))
print("\nПропуски:")
print(anime_df.isnull().sum())
print("\nДубликаты:", anime_df.duplicated().sum())

print("\nUsers Score DF:")
print("Shape:", users_score_df.shape)
users_score_df.info()
print("\nБазовые статистики (числовые):")
print(users_score_df.describe())
print("\nКатегориальные статистики:")
print(users_score_df.describe(include='object'))
print("\nПропуски:")
print(users_score_df.isnull().sum())
print("\nДубликаты:", users_score_df.duplicated().sum())

print("\nUsers Details DF:")
print("Shape:", users_details_df.shape)
users_details_df.info()
print("\nБазовые статистики (числовые):")
print(users_details_df.describe())
print("\nКатегориальные статистики:")
print(users_details_df.describe(include='object'))
print("\nПропуски:")
print(users_details_df.isnull().sum())
print("\nДубликаты:", users_details_df.duplicated().sum())

print("\nMerged DF:")
print("Shape:", merged_scores_details.shape)
merged_scores_details.info()
print("\nБазовые статистики (числовые, выборка 100000 строк):")
print(merged_scores_details.sample(n=100000).describe())
print("\nКатегориальные статистики (выборка 100000 строк):")
print(merged_scores_details.sample(n=100000).describe(include='object'))
print("\nПропуски:")
print(merged_scores_details.isnull().sum())
print("\nДубликаты:", merged_scores_details.duplicated().sum())

=== Статистика после предобработки ===

Anime DF:
Shape: (24905, 24)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24905 entries, 0 to 24904
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   anime_id      24905 non-null  int64         
 1   Name          24905 non-null  object        
 2   English name  24905 non-null  object        
 3   Other name    24905 non-null  object        
 4   Score         24905 non-null  float64       
 5   Genres        24905 non-null  object        
 6   Synopsis      24905 non-null  object        
 7   Type          24905 non-null  object        
 8   Episodes      24905 non-null  float64       
 9   Aired         20090 non-null  datetime64[ns]
 10  Premiered     24905 non-null  object        
 11  Status        24905 non-null  object        
 12  Producers     24905 non-null  object        
 13  Licensors     24905 non-null  object        
 14  Studios       249

In [21]:
# Сохранение всех обработанных DataFrame в папку processed
anime_df.to_csv(processed_anime_path, index=False)
print(f"Сохранен anime_processed.csv в {processed_anime_path}")

users_score_df.to_csv(processed_scores_path, index=False)
print(f"Сохранен users_score_processed.csv в {processed_scores_path}")

# users_details_df – сохраняем даты в ISO-формате
users_details_df.to_csv(
    processed_details_path,
    index=False,
    date_format='%Y-%m-%d'          # только дата, без времени
)
print(f"Сохранен users_details_processed.csv в {processed_details_path}")

# merged_scores_details – тоже даты в ISO
merged_scores_details.to_csv(
    merged_path,
    index=False,
    date_format='%Y-%m-%d'
)
print(f"Сохранен merged_scores_details.csv в {merged_path}")

Сохранен anime_processed.csv в ../data/processed\anime_processed.csv
Сохранен users_score_processed.csv в ../data/processed\users_score_processed.csv
Сохранен users_details_processed.csv в ../data/processed\users_details_processed.csv
Сохранен merged_scores_details.csv в ../data/processed\merged_scores_details.csv
